# Demonstration of next-generation seaborn interface

## The basic interface

The new interface exists as a set of classes that can be acessed through a single namespace import:

In [ ]:
import seaborn.objects as so

This is a clean namespace, and I'm leaning towards recommending `from seaborn.objects import *` for interactive usecases. But let's not go so far just yet.

Let's also import the main namespace so we can load our trusty example datasets.

In [ ]:
import seaborn
seaborn.set_theme()

The main object is `seaborn.objects.Plot`. You instantiate it by passing data and some assignments from columns in the data to roles in the plot:

In [ ]:
tips = seaborn.load_dataset("tips")
so.Plot(tips, x="total_bill", y="tip")

But instantiating the `Plot` object doesn't actually plot anything. For that you need to add some layers:

In [ ]:
so.Plot(tips, x="total_bill", y="tip").add(so.Scatter())

Variables can be defined globally, or for a specific layer:

In [ ]:
so.Plot(tips).add(so.Scatter(), x="total_bill", y="tip")

Each layer can also have its own data:

In [ ]:
(
    so.Plot(tips, x="total_bill", y="tip")
    .add(so.Scatter(color=".6"), data=tips.query("size != 2"))
    .add(so.Scatter(), data=tips.query("size == 2"))
)

As in the existing interface, variables can be keys to the `data` object or vectors of various kinds:

In [ ]:
(
    so.Plot(tips.to_dict(), x="total_bill")
    .add(so.Scatter(), y=tips["tip"].to_numpy())
)

The interface also supports semantic mappings between data and plot variables. But the specification of those mappings uses more explicit parameter names:

In [ ]:
so.Plot(tips, x="total_bill", y="tip", color="time").add(so.Scatter())

It also offers a wider range of mappable features:

In [ ]:
(
    so.Plot(tips, x="total_bill", y="tip", color="day", fill="time")
    .add(so.Scatter(fillalpha=.8))
)

---

## Core components

### Visual representation: the Mark

Each layer needs a `Mark` object, which defines how to draw the plot. There will be marks corresponding to existing seaborn functions and ones offering new functionality. But not many have been implemented yet:

In [ ]:
fmri = seaborn.load_dataset("fmri").query("region == 'parietal'")
so.Plot(fmri, x="timepoint", y="signal").add(so.Line())

`Mark` objects will expose an API to set features directly, rather than mapping them:

In [ ]:
so.Plot(tips, y="day", x="total_bill").add(so.Dot(color="#698", alpha=.5))

### Data transformation: the Stat


Built-in statistical transformations are one of seaborn's key features. But currently, they are tied up with the different visual representations. E.g., you can aggregate data in `lineplot`, but not in `scatterplot`.

In the new interface, these concerns are separated. Each layer can accept a `Stat` object that applies a data transformation:

In [ ]:
so.Plot(fmri, x="timepoint", y="signal").add(so.Line(), so.Agg())

The `Stat` is computed on subsets of data defined by the semantic mappings:

In [ ]:
so.Plot(fmri, x="timepoint", y="signal", color="event").add(so.Line(), so.Agg())

Each mark also accepts a `group` mapping that creates subsets without altering visual properties:

In [ ]:
(
    so.Plot(fmri, x="timepoint", y="signal", color="event")
    .add(so.Line(), so.Agg(), group="subject")
)

The `Mark` and `Stat` objects allow for more compositionality and customization. There will be guidelines for how to define your own objects to plug into the broader system:

In [ ]:
class PeakAnnotation(so.Mark):
    def plot(self, split_generator, scales, orient):
        for keys, data, ax in split_generator():
            ix = data["y"].idxmax()
            ax.annotate(
                "The peak", data.loc[ix, ["x", "y"]],
                xytext=(10, -100), textcoords="offset points",
                va="top", ha="center",
                arrowprops=dict(arrowstyle="->", color=".2"),

            )

(
    so.Plot(fmri, x="timepoint", y="signal")
    .add(so.Line(), so.Agg())
    .add(PeakAnnotation(), so.Agg())
)

The new interface understands not just `x` and `y`, but also range specifiers; some `Stat` objects will output ranges, and some `Mark` objects will accept them. (This means that it will finally be possible to pass pre-defined error-bars into seaborn):

In [ ]:
(
    fmri
    .groupby("timepoint")
    .signal
    .describe()
    .pipe(so.Plot, x="timepoint")
    .add(so.Line(), y="mean")
    .add(so.Ribbon(alpha=.2), ymin="min", ymax="max")
)

-----

### Overplotting resolution: the Move

Existing seaborn functions have parameters that allow adjustments for overplotting, such as `dodge=` in several categorical functions, `jitter=` in several functions based on scatterplots, and the `multiple=` paramter in distribution functions. In the new interface, those adjustments are abstracted away from the particular visual representation into the concept of a `Move`:

In [ ]:
(
    so.Plot(tips, "day", "total_bill", color="time")
    .add(so.Bar(), so.Agg(), move=so.Dodge())
)

Separating out the positional adjustment makes it possible to add additional flexibility without overwhelming the signature of a single function. For example, there will be more options for handling missing levels when dodging and for fine-tuning the adjustment.

In [ ]:
(
    so.Plot(tips, "day", "total_bill", color="time")
    .add(so.Bar(), so.Agg(), move=so.Dodge(empty="fill", gap=.1))
)

By default, the `move` will resolve all overlapping semantic mappings:

In [ ]:
(
    so.Plot(tips, "day", "total_bill", color="time", alpha="sex")
    .add(so.Bar(), so.Agg(), move=so.Dodge())
)

But you can specify a subset:

In [ ]:
(
    so.Plot(tips, "day", "total_bill", color="time", alpha="smoker")
    .add(so.Dot(), move=so.Dodge(by=["color"]))
)

It's also possible to stack multiple moves or kinds of moves by passing a list:

In [ ]:
(
    so.Plot(tips, "day", "total_bill", color="time", alpha="smoker")
    .add(
        so.Dot(),
        move=[so.Dodge(by=["color"]), so.Jitter(.5)]
    )
)

Separating the `Stat` and `Move` from the visual representation affords more flexibility, greatly expanding the space of graphics that can be created.

-----

### Semantic mapping: the Scale

The declarative interface allows users to represent dataset variables with visual properites such as position, color or size. A complete plot can be made without doing anything more defining the mappings: users need not be concerned with converting their data into units that matplotlib understands. But what if one wants to alter the mapping that seaborn chooses? This is accomplished through the concept of a `Scale`.

The notion of scaling will probably not be unfamiliar; as in matplotlib, seaborn allows one to apply a mathematical transformation, such as `log`, to the coordinate variables:

In [ ]:
planets = seaborn.load_dataset("planets").query("distance < 1000")

In [ ]:
(
    so.Plot(planets, x="mass", y="distance")
    .scale(x="log", y="log")
    .add(so.Scatter())
)

But the `Scale` concept is much more general in seaborn: a scale can be provided for any mappable property. For example, it is how you specify the palette used for color variables:

In [ ]:
(
    so.Plot(planets, x="mass", y="distance", color="orbital_period")
    .scale(x="log", y="log", color="rocket")
    .add(so.Scatter())
)

While there are a number of short-hand "magic" arguments you can provide for each scale, it is also possible to be more explicit by passing a `Scale` object. There are several distinct `Scale` classes, corresponding to the fundamental scale types (nominal, ordinal, continuous, etc.). Each class exposes a number of relevant parameters that control the details of the mapping:

In [ ]:
(
    so.Plot(planets, x="mass", y="distance", color="orbital_period")
    .scale(
        x="log",
        y=so.Continuous(transform="log").tick(at=[3, 10, 30, 100, 300]),
        color=so.Continuous("rocket", transform="log"),
    )
    .add(so.Scatter())
)

There are several different kinds of scales, including scales appropriate for categorical data:

In [ ]:
(
    so.Plot(planets, x="year", y="distance", color="method")
    .scale(
        y="log",
        color=so.Nominal(["b", "g"], order=["Radial Velocity", "Transit"])
    )
    .add(so.Scatter())
)

It's also possible to disable scaling for a variable so that the literal values in the dataset are passed directly through to matplotlib:

In [ ]:
(
    so.Plot(planets, x="distance", y="orbital_period", pointsize="mass")
    .scale(x="log", y="log", pointsize=None)
    .add(so.Scatter())
)

Scaling interacts with the `Stat` and `Move` transformations. When an axis has a nonlinear scale, any statistical transformations or adjustments take place in the appropriate space:

In [ ]:
so.Plot(planets, x="distance").add(so.Bar(), so.Hist()).scale(x="log")

This is also true of the `Move` transformations:

In [ ]:
(
    so.Plot(
        planets, x="distance",
        color=(planets["number"] > 1).rename("multiple")
    )
    .add(so.Bar(), so.Hist(), so.Dodge())
    .scale(x="log", color=so.Nominal())
)

----

## Defining subplot structure

Seaborn's faceting functionality (drawing subsets of the data on distinct subplots) is built into the `Plot` object and works interchangably with any `Mark`/`Stat`/`Move`/`Scale` spec:

In [ ]:
(
    so.Plot(tips, x="total_bill", y="tip")
    .facet("time", order=["Dinner", "Lunch"])
    .add(so.Scatter())
)

Unlike the existing `FacetGrid` it is simple to *not* facet a layer, so that a plot is simply replicated across each column (or row):

In [ ]:
(
    so.Plot(tips, x="total_bill", y="tip")
    .facet(col="day")
    .add(so.Scatter(color=".75"), col=None)
    .add(so.Scatter(), color="day")
    .configure(figsize=(7, 3))
)

The `Plot` object *also* subsumes the `PairGrid` functionality:

In [ ]:
(
    so.Plot(tips, y="day")
    .pair(x=["total_bill", "tip"])
    .add(so.Dot())
)

Pairing and faceting can be combined in the same plot:

In [ ]:
(
    so.Plot(tips, x="day")
    .facet("sex")
    .pair(y=["total_bill", "tip"])
    .add(so.Dot())
)

Or the `Plot.pair` functionality can be used to define unique pairings between variables:

In [ ]:
(
    so.Plot(tips)
    .pair(x=["day", "time"], y=["total_bill", "tip"], cross=False)
    .add(so.Dot())
)

It's additionally possible to "pair" with a single variable, for univariate plots like histograms.

Both faceted and paired plots with subplots along a single dimension can be "wrapped", and this works both columwise and rowwise:

In [ ]:
(
    so.Plot(tips)
    .pair(x=tips.columns, wrap=3)
    .configure(sharey=False)
    .add(so.Bar(), so.Hist())
)

Importantly, there's no distinction between "axes-level" and "figure-level" here. Any kind of plot can be faceted or paired by adding a method call to the `Plot` definition, without changing anything else about how you are creating the figure.

---

## Iterating and displaying

It is possible (and in fact the deafult behavior) to be completely pyplot-free, and all the drawing is done by directly hooking into Jupyter's rich display system. Unlike in normal usage of the inline backend, writing code in a cell to define a plot is indendent from showing it:

In [ ]:
p = so.Plot(fmri, x="timepoint", y="signal").add(so.Line(), so.Agg())

In [ ]:
p

By default, the methods on `Plot` do *not* mutate the object they are called on. This means that you can define a common base specification and then iterate on different versions of it.

In [ ]:
p = (
    so.Plot(fmri, x="timepoint", y="signal", color="event")
    .scale(color="crest")
)

In [ ]:
p.add(so.Line())

In [ ]:
p.add(so.Line(), group="subject")

In [ ]:
p.add(so.Line(), so.Agg())

In [ ]:
(
    p
    .add(so.Line(linewidth=.5, alpha=.5), group="subject")
    .add(so.Line(linewidth=3), so.Agg())
)

It's also possible to hook into the `pyplot` system by calling `Plot.show`. (As you might in a terminal interface, or to use a GUI). Notice how this looks lower-res: that's because `Plot` is generating "high-DPI" figures internally!

In [ ]:
(
    p
    .add(so.Line(linewidth=.5, alpha=.5), group="subject")
    .add(so.Line(linewidth=3), so.Agg())
    .show()
)

---

## Matplotlib integration

It's always been a design aim in seaborn to allow complicated seaborn plots to coexist within the context of a larger matplotlib figure. This is acheived within the "axes-level" functions, which accept an `ax=` parameter. The `Plot` object *will* provide a similar functionality:

In [ ]:
import matplotlib as mpl
_, ax = mpl.figure.Figure(constrained_layout=True).subplots(1, 2)
(
    so.Plot(tips, x="total_bill", y="tip")
    .on(ax)
    .add(so.Scatter())
)

But a limitation has been that the "figure-level" functions, which can produce multiple subplots, cannot be directed towards an existing figure. That is no longer the case; `Plot.on()` also accepts a `Figure` (created either with or without `pyplot`) object:

In [ ]:
f = mpl.figure.Figure(constrained_layout=True)
(
    so.Plot(tips, x="total_bill", y="tip")
    .on(f)
    .add(so.Scatter())
    .facet("time")
)

Providing an existing figure is perhaps only marginally useful. While it will ease the integration of seaborn with GUI frameworks, seaborn is still using up the whole figure canvas. But with the introduction of the `SubFigure` concept in matplotlib 3.4, it becomes possible to place a small-multiples plot *within* a larger set of subplots:

In [ ]:
f = mpl.figure.Figure(constrained_layout=True, figsize=(8, 4))
sf1, sf2 = f.subfigures(1, 2)
(
    so.Plot(tips, x="total_bill", y="tip", color="day")
    .add(so.Scatter())
    .on(sf1)
    .plot()
)
(
    so.Plot(tips, x="total_bill", y="tip", color="day")
    .facet("day", wrap=2)
    .add(so.Scatter())
    .on(sf2)
    .plot()
)